In [1]:
import streamlit as st
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("Export_TER_juin2023_FIX_SafwanChendeb.csv",sep=";")
print(df.info)

DATA CLEAN

Enlever les valeurs nulles ou trop petites de Largeur de bande

In [4]:
df2 = df.dropna(subset=['ASS_LGBD_KHZ'])
df2 = df2[df2["ASS_LGBD_KHZ"] >1000]


Enlever les antennes qui ont été supprimées

In [14]:
enreg_to_supp = df2[df2['MVT_CODE'] == 'SUP']['N° ENREG']
df2 = df2[~df2['N° ENREG'].isin(enreg_to_supp)]


Garder seulement les dernières modifications de chaque antenne

In [15]:
df2['Date CAF'] = pd.to_datetime(df['Date CAF'], format='%d/%m/%Y')
df2 = df2.sort_values('Date CAF').drop_duplicates(subset = "N° ENREG", keep = 'last')

TEST

In [ ]:
df2.head(20)